# 필수 라이브러리 로딩

In [1]:
# 필수 라이브러리 로딩
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import csv
import time
import re # 숫자만 남기려구

import pandas as pd

# 필요한 정보
- 상품명(pname), 총별점(tstar), 닉네임(name), 별점(star), 작성일(pdate), 기타리뷰(rv)
- pname_l , ..., non_l(리뷰없음), fail_l(경로가 다른지 안가져와진 애들)
## 경로 설정

In [2]:
# 화면 주소 정의
## 스킨케어
a = 'https://www.oliveyoung.co.kr/store/display/getCategoryShop.do?dispCatNo=10000010001'
## 마스크팩
b = 'https://www.oliveyoung.co.kr/store/display/getCategoryShop.do?dispCatNo=10000010009&gateCd=Drawer'
## 클렌징
c = 'https://www.oliveyoung.co.kr/store/display/getCategoryShop.do?dispCatNo=10000010010&gateCd=Drawer'
## 선케어
d = 'https://www.oliveyoung.co.kr/store/display/getCategoryShop.do?dispCatNo=10000010011&gateCd=Drawer'

## 사용자 함수 정의

In [3]:
# 리뷰건수 계산하는 함수
## return을 활용한 전역변수화
def rv_num():
    q = driver.find_element(By.XPATH, '//*[@id="gdasContentsArea"]/div/div[2]/div/div[2]/p[1]/em').text
    q = int(re.sub(r"[^0-9]", "", q)) # 정수
    if q == 0:
        pn = 'zero'
        pc = 'zero'
    elif q <= 10:
        pn = 'zero'
        pc = q
    else:   
    # page_num(리뷰 페이지 수), product_count(마지막페이지의 리뷰수)
        pn = (q // 10) + 1 
        pc = q % 10
        
    return pn, pc

### 사용시 pn, pc = rv_num()

In [4]:
# 총별점(str로 저장됨), 상품명 
## global 사용
def rv_info():
    global pname
    pname = driver.find_element(By.XPATH,'//*[@id="Contents"]/div[2]/div[2]/div/p[2]').text
    global tstar
    tstar = driver.find_element(By.CSS_SELECTOR,'#gdasContentsArea > div > div.product_rating_area.review-write-delete > div > div.star_area > p.num > strong').text
    
    

In [5]:
# 필요한 정보들 리스트업 하는 함수
def list_up(n=10):  # 파라미터는 한 페이지 안에 있는 리뷰수
    for i in range(1, n+1):
        # 닉네임
        y = driver.find_element(By.XPATH, f'//*[@id="gdasList"]/li[{i}]').text
        name = y.split(sep='\n')[0]
        name_l.append(name)
        # 리뷰 블럭
        x = driver.find_element(By.XPATH, f'//*[@id="gdasList"]/li[{i}]/div[2]').text
        rv = x.split(sep='\n')
        ## 별점
        star = rv[0]
        ## 작성일
        pdate = rv[1]
        ##기타
        del rv[0:2] 
        del rv[-2:]
        rv_l.append(rv)  # 요소 추가
        pname_l.append(pname)
        tstar_l.append(tstar)
        name_l.append(name)
        star_l.append(star)
        pdate_l.append(pdate)
        print(f'_{i}')   

In [6]:
# 한가지 상품안에서 돌아가는 구문 함수
def one_product(pn, pc):
    list_up() # 1페이지 리뷰 가져오기
    for l in range(2, pn+2):
        if str(l)[-1] == '0':
            driver.find_element(By.LINK_TEXT, f'{l}').click()
            driver.implicitly_wait(3)
            # 10개의 정보 가져오는 함수 기본함수정의해서파라미터로 숫자넣기(10) 
            list_up()
            driver.find_element(By.PARTIAL_LINK_TEXT, f'다음').click()
            driver.implicitly_wait(3)
            print(l)
            list_up()
            # 함수가져오기 list_up()
        elif str(l)[-1] == '0' and l == pn: # 다음페이지가 마지막인 경우
            driver.find_element(By.LINK_TEXT, f'{l}').click()
            driver.implicitly_wait(3)
            list_up()
            driver.find_element(By.PARTIAL_LINK_TEXT, f'다음').click()
            driver.implicitly_wait(3)
            print(l)
            if pc != 0: # pc가 0이면 for문 안돌아가서 
                list_up(pc)
            else:
                list_up()  
                
        elif str(l)[-1] == '1':
            print(l)
            
        elif l == (pn+1): # 마지막 페이지
            driver.find_element(By.LINK_TEXT, f'{l}').click()
            driver.implicitly_wait(3)
            if pc != 0:
                list_up(pc)
            else:
                list_up()
            # 끝자리수만큼 포문 pc list_up(pc)
            print(l)
        else:
            driver.find_element(By.LINK_TEXT, f'{l}').click()
            driver.implicitly_wait(3)
            list_up() # 까먹었었음 ㅜㅜ
            print(l)
    

In [7]:
# 위에 함수 예외처리 버전 함수
def try_pro():
    try:
        pn, pc = rv_num()
        rv_info()
        if pn == 'zero' and pc == 'zero':
            # 상품리뷰 없음. 
            non_l.append(pname) # 상품명 리스트에는 안올라감
        elif pn == 'zero':
            list_up(pc)
        else:
            one_product(pn, pc)
    except:
        pname = driver.find_element(By.XPATH,'//*[@id="Contents"]/div[2]/div[2]/div/p[2]').text
        fail_l.append(pname)
        print("##### 다른 양식인가봐..#####", pname)

## 실전
### 드라이버 실행

In [8]:
URL = 'https://www.oliveyoung.co.kr/store/main/main.do?oy=0' # 첫화면
ops = webdriver.ChromeOptions()
ops.add_experimental_option('excludeSwitches', ['enable-logging'])

# chromedriver path
sv = Service(executable_path= 'C:\\Users\\admin\\Downloads\\chroemdriver')
driver = webdriver.Chrome(service=sv, options=ops)

In [9]:
driver.get(url=a)
driver.implicitly_wait(3)
# 기본페이지 이동
# 스킨케어_토너 페이지로 이동
driver.find_element(By.XPATH, f'//*[@id="Contents"]/div/div[1]/ul/li[1]/a').click()
driver.implicitly_wait(3)
# 리뷰 48개로 선택
driver.find_element(By.XPATH, '//*[@id="Contents"]/div[5]/div[2]/ul/li[3]/a').click()
driver.implicitly_wait(3)

In [10]:
# # 한제품 안에서 돌아가는 부분
# for l in range(2, pn+2): 
#     if str(l)[-1] == '0':
#         driver.find_element(By.LINK_TEXT, f'{l}').click()
#         driver.implicitly_wait(3)
#         # 10개의 정보 가져오는 함수 기본함수정의해서파라미터로 숫자넣기(10) 
#         list_up()
#         driver.find_element(By.PARTIAL_LINK_TEXT, f'다음').click()
#         driver.implicitly_wait(3)
#         print(l)
#         list_up()
#         # 함수가져오기 list_up()
#     elif str(l)[-1] == '0' and l == pn: # 다음페이지가 마지막인 경우
#         driver.find_element(By.LINK_TEXT, f'{l}').click()
#         driver.implicitly_wait(3)
#         list_up()
#         driver.find_element(By.PARTIAL_LINK_TEXT, f'다음').click()
#         driver.implicitly_wait(3)
#         print(l)
#         if pc != 0: # pc가 0이면 for문 안돌아가서 
#             list_up(pc)
#         else:
#             list_up()  
#     elif str(l)[-1] == '1':
#         print(l)
#     elif l == (pn+1): # 마지막 페이지
#         driver.find_element(By.LINK_TEXT, f'{i}').click()
#         driver.implicitly_wait(3)
#         if pc != 0:
#             list_up(pc)
#         else:
#             list_up()
#         # 끝자리수만큼 포문 pc list_up(pc)
#         print(l)
#     else:
#         driver.find_element(By.LINK_TEXT, f'{i}').click()
#         driver.implicitlt_wait(3)
#         print(l)

In [10]:
## 11번째로 이동
driver.find_element(By.PARTIAL_LINK_TEXT, f'다음').click()
driver.implicitly_wait(3)

In [11]:
# 리스트 정의
pname_l = []
tstar_l = []
name_l = []
star_l = []
pdate_l = []
rv_l = []

non_l = []
fail_l = []

In [12]:
# # 48.1
# try:
#     for i in range(1, 13):
#         for j in range(1,5):
#             # 상품 선택
#             driver.find_element(By.XPATH,f'//*[@id="Contents"]/ul[{i+1}]/li[{j}]/div/a/img').click()
#             time.sleep(3)
#             # 리뷰칸 선택
#             driver.find_element(By.XPATH,f'//*[@id="reviewInfo"]/a').click()
#             time.sleep(3)
#             try_pro()
#             driver.back()
# except:
#     print('어디서 오류가 났으려나? 일단 pass^^')

In [12]:
# for문 가능?
for x in range(2,7):
    try:
        for i in range(1, 13):
            for j in range(1,5):
                # 상품 선택
                driver.find_element(By.XPATH,f'//*[@id="Contents"]/ul[{i+1}]/li[{j}]/div/a/img').click()
                time.sleep(3)
                # 리뷰칸 선택
                driver.find_element(By.XPATH,f'//*[@id="reviewInfo"]/a').click()
                time.sleep(3)
                try_pro()
                driver.back()
    except:
        print('어디서 오류가 났으려나? 일단 pass^^')
    try:
        driver.find_element(By.LINK_TEXT, f'{10+x}').click()
        driver.implicitly_wait(3)
    except:
        continue

##### 다른 양식인가봐..##### 허브노트 아하 바하 파하 리프레쉬 퓨리파잉 토너 200ml
##### 다른 양식인가봐..##### 싸이닉 히아루론산 앰플 스킨 300ml
##### 다른 양식인가봐..##### 잇츠스킨 레몬씨 스퀴즈 앰플 토너 500ml
##### 다른 양식인가봐..##### 디오디너리 내추럴 모이스처라이징 팩터스+에이치에이 30ml
##### 다른 양식인가봐..##### 라네즈옴므 크림 스킨 옴므 올인원 150ml
##### 다른 양식인가봐..##### 클럽맨리저브 브랜디스파이스 에프터쉐이브 50ml
##### 다른 양식인가봐..##### 마녀공장 올인원 데일리 로션 포맨 100ml
##### 다른 양식인가봐..##### 갸스비 포어 케어 올인원 에센스 145ml
##### 다른 양식인가봐..##### 라네즈옴므 액티브워터 스킨 180ml
##### 다른 양식인가봐..##### 라네즈옴므 액티브워터 모이스처라이저 125ml
_1
_2
_3
_4
_5
_6
_7
_8
_9
_10
_1
_2
_3
_4
_5
_6
_7
_8
_9
_10
2
_1
_2
_3
_4
_5
_6
_7
_8
_9
_10
3
_1
_2
_3
_4
_5
_6
_7
_8
_9
_10
4
_1
_2
_3
_4
_5
_6
_7
_8
_9
_10
5
_1
_2
_3
_4
_5
_6
_7
_8
_9
_10
6
_1
_2
_3
_4
_5
_6
_7
_8
_9
_10
7
_1
_2
_3
_4
_5
_6
_7
_8
_9
_10
8
_1
_2
_3
_4
_5
_6
_7
_8
_9
_10
9
_1
_2
_3
##### 다른 양식인가봐..##### 아벤느 트릭세라 더블 기획 (100ml+100ml+젠틀토닝로션100ml 증정)
##### 다른 양식인가봐..##### 한율 빨간쌀 진액스킨 150ml
##### 다른 양식인가봐..##### 바르테라 올인원 젤 모이스춰라이저
_1
_2
_3
_4
_5
_6
_7
_8
_9
_10
_1
_2
_3
_4
_5
_6
_7
_8
_9
_10
2
_1
_

In [13]:
# 데이터 프레임 key들끼리 행 길이 같아야 함
try:   
    rv_rv = pd.DataFrame({'pname':pname_l, 'tstar':tstar_l, 'name':name_l,\
                          'star':star_l,'pdate':pdate_l, 'rv':rv_l})
    rv_rv.to_csv('./newtry_rv3.csv', sep = ',', encoding = 'utf-8-sig')
except:
    rv_rv = pd.DataFrame({'star':star_l,'pdate':pdate_l, 'rv':rv_l})
    rv_rv.to_csv('./newtry_rv3.csv', sep = ',', encoding = 'utf-8-sig')
    rv_name = pd.DataFrame({'pname':pname_l, 'tstar':tstar_l})
    rv_name.to_csv('./newtry_name3.csv', sep = ',', encoding = 'utf-8-sig')
    print('this')
    

this


In [14]:
rv_zero = pd.DataFrame({'pname':non_l})
rv_zero.to_csv('./newtry_zero3.csv', sep = ',', encoding = 'utf-8-sig')
rv_fail = pd.DataFrame({'pname':fail_l})
rv_fail.to_csv('./newtry_fail3.csv', sep = ',', encoding = 'utf-8-sig')

In [15]:
# len(star_l)
# # 'pname':pname_l, 'tstar':tstar_l, 'name':name_l,\ # 길이가 달라서 저장 못함 
# rv_name = pd.DataFrame({'pname':pname_l, 'tstar':tstar_l})
# rv_name.to_csv('./newtry_name.csv', sep = ',', encoding = 'utf-8-sig')
driver.close()